In [1]:
import os
import tensorflow as tf
import numpy as np
import pathlib
from tqdm import tqdm
import platform

In [2]:
SYS_SEP = '\\' if platform.system()=='Windows' else '/'
PATH = "../data/train"
# os.chdir(path)
directory = os.listdir(PATH)
list_image, list_labels = [], []


In [3]:
def process_path(file_path):
    parts = tf.strings.split(file_path, SYS_SEP)

    return file_path, int(parts[-2] == b'Positive')

def load_dataset(data_root="../data/" ):
    data_root = pathlib.Path(data_root)
    list_dirs = [str(data_root/'*/*.jpg'),str(data_root/'*/*.png')]
    
    list_ds = tf.data.Dataset.list_files(list_dirs)
    
    SIZE = len(list(list_ds))
    labeled_ds = list_ds.map(process_path)
      
    return labeled_ds, SIZE


In [4]:
labeled_ds,SIZE = load_dataset(PATH)


In [5]:
IMAGE_SIZE = 64
    
for img, classe in tqdm(labeled_ds):
    image = tf.io.read_file(img)
    image = tf.image.decode_image(image, channels=1)

    # Adjusting Contrast 
    image = tf.image.resize_with_pad(image,target_height=IMAGE_SIZE,
                                     target_width=IMAGE_SIZE,
                                     antialias=True,method='gaussian')

    image = tf.image.adjust_contrast(image, contrast_factor=2.91)

    image = tf.cast(image, tf.float32)
    image = tf.math.divide(image,255.0)  # normalize to [0,1] range

    list_image.append(image.numpy())
    list_labels.append(classe)



10357it [00:36, 287.29it/s]


In [8]:
x = np.asarray(list_image)
y = np.asarray(list_labels)

np.save(os.path.join(PATH,"data-images.npy"), x)
np.save(os.path.join(PATH,"data-labels.npy"), y)


print(PATH+"data-images.npy\n"+PATH+"data-labels.npy\n"+"Done...!")


../data/traindata-images.npy
../data/traindata-labels.npy
Done...!


In [9]:
x.shape

(10357, 64, 64, 1)